In [1]:
import os
import pandas as pd
csv_path= "D:/final_speech/Dataset/Recordings/audio__details.csv"
audio_folder_path = "D:/final_speech/Dataset/Recordings/Train"
df= pd.read_csv(csv_path)
df['File_name'] = df['File_name'].apply(lambda x: os.path.abspath(os.path.join(audio_folder_path, os.path.basename(x))))
df.to_csv(csv_path, index=False)

In [2]:
from datasets import load_dataset, Audio
dataset = load_dataset('csv',data_files="D:/final_speech/Dataset/Recordings/audio__details.csv")
dataset = dataset.cast_column('File_name',Audio(sampling_rate=16000))
dataset = dataset.rename_column('File_name',"audio")
dataset = dataset.rename_column("phrase",'sentence')

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base")

def prepare_dataset(batch):
    # Process audio to input features
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    # Process transcription to labels
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch
# Apply the processing function to the dataset
dataset = dataset.map(prepare_dataset)

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [4]:
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.nn.utils.rnn import pad_sequence


model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")


training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=False,
    save_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
)

E:\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
class DataCollatorForWhisper:
    def __call__(self, features):
        
        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        labels = [torch.tensor(feature["labels"]) for feature in features]

        
        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_features": input_features_padded,
            "labels": labels_padded
        }

In [6]:
data_collator = DataCollatorForWhisper()

In [7]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=processor.tokenizer,  
    data_collator=data_collator,
)

In [8]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


E:\Lib\site-packages\transformers\modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=9, training_loss=4.482733832465278, metrics={'train_runtime': 134.1192, 'train_samples_per_second': 2.416, 'train_steps_per_second': 0.067, 'total_flos': 1.81607989248e+16, 'train_loss': 4.482733832465278, 'epoch': 2.571428571428571})

In [9]:
model.save_pretrained("whisper_finetuned")
processor.save_pretrained("whisper_finetuned")

[]

In [14]:
import whisper
from jiwer import wer
from transformers import pipeline
# Load the fine-tuned model
model = pipeline("automatic-speech-recognition", model="whisper-finetuned")
# Transcribe the audio file
result1 = model("D:/final_speech/Dataset/Recordings/Train/1.mp3")
prediction1 = result1["text"]
print(prediction1)
hypothesis1= "learning new languauges is fun"
error1 = wer(prediction1, hypothesis1)
print("WER 1 is:",error1)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


 Learning new language is fun.
WER 1 is: 0.6
